# Mean-Variance vs Kelly Criterion

This notebook provides a deep dive comparing Markowitz mean-variance optimization with the Kelly criterion.

## Key Differences

1. **Mean-Variance (Markowitz 1952)**
   - Single-period framework
   - Maximizes E[R] - (λ/2)Var[R] for risk aversion λ
   - Yields efficient frontier in mean-std space
   - Optimal for quadratic utility or normal returns

2. **Kelly Criterion (Kelly 1956)**
   - Multi-period/infinite horizon framework
   - Maximizes E[log(1+R)] - geometric mean growth
   - Growth-optimal: maximizes long-run wealth almost surely
   - Corresponds to log utility

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

from portfolio_gambling.single_period import SinglePeriodOptimizer
from portfolio_gambling.multi_period import MultiPeriodOptimizer
from portfolio_gambling.utils import generate_returns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print("Mean-Variance vs Kelly Criterion Comparison")
print("=" * 50)

## 1. Generate Market Data

Create 5 assets with different risk-return profiles.

In [ ]:
# Generate synthetic returns
n_assets = 5
n_periods = 1000

# Create diverse asset universe
mean_returns = np.array([0.10, 0.12, 0.08, 0.15, 0.06]) / 252  # Daily returns
volatilities = np.array([0.15, 0.25, 0.12, 0.30, 0.10]) / np.sqrt(252)

# Correlation matrix
corr = np.array([
    [1.0, 0.6, 0.4, 0.3, 0.2],
    [0.6, 1.0, 0.5, 0.7, 0.3],
    [0.4, 0.5, 1.0, 0.4, 0.6],
    [0.3, 0.7, 0.4, 1.0, 0.2],
    [0.2, 0.3, 0.6, 0.2, 1.0]
])

# Covariance matrix
cov_matrix = np.outer(volatilities, volatilities) * corr

# Generate returns
returns = generate_returns(n_assets, n_periods, mean_returns, cov_matrix)

print(f"Generated {n_periods} periods of returns for {n_assets} assets")
print(f"\nAnnualized Statistics:")
print(f"Mean returns: {mean_returns * 252 * 100}")
print(f"Volatilities: {volatilities * np.sqrt(252) * 100}")

## 2. Compute Optimal Portfolios

Calculate both mean-variance and Kelly optimal portfolios.

In [ ]:
# Initialize optimizers
sp_opt = SinglePeriodOptimizer(returns)
mp_opt = MultiPeriodOptimizer(returns)

# Mean-variance optimization (tangency/max Sharpe)
mv_weights = sp_opt.mean_variance()

# Kelly criterion
kelly_weights = mp_opt.kelly_criterion()

# Half Kelly (more conservative)
half_kelly_weights = mp_opt.fractional_kelly(0.5)

# Equal weight for comparison
equal_weights = np.ones(n_assets) / n_assets

# Display weights
weights_df = pd.DataFrame({
    'Mean-Variance': mv_weights,
    'Kelly': kelly_weights,
    'Half Kelly': half_kelly_weights,
    'Equal Weight': equal_weights
}, index=[f'Asset {i+1}' for i in range(n_assets)])

print("\nOptimal Portfolio Weights:")
print(weights_df.round(4))
print(f"\nSum of weights:")
print(weights_df.sum())

## 3. Compare Single-Period Statistics

In [ ]:
# Calculate single-period metrics
strategies = {
    'Mean-Variance': mv_weights,
    'Kelly': kelly_weights,
    'Half Kelly': half_kelly_weights,
    'Equal Weight': equal_weights
}

metrics_list = []
for name, weights in strategies.items():
    metrics = sp_opt.portfolio_metrics(weights)
    metrics['strategy'] = name
    metrics_list.append(metrics)

metrics_df = pd.DataFrame(metrics_list).set_index('strategy')
print("\nSingle-Period Metrics (Annualized):")
print(metrics_df[['expected_return', 'volatility', 'sharpe_ratio']].round(4) * np.array([252, np.sqrt(252), np.sqrt(252)]))

## 4. Multi-Period Simulations

The key difference emerges in multi-period performance.

In [ ]:
# Simulate strategies over time
n_sim_periods = 252  # 1 year
n_simulations = 1000

results = {}
for name, weights in strategies.items():
    results[name] = mp_opt.simulate_strategy(weights, n_sim_periods, n_simulations)

# Display results
print("\nMulti-Period Simulation Results (1 year, 1000 paths):")
for name, res in results.items():
    print(f"\n{name}:")
    print(f"  Mean final wealth: ${res['mean_final_wealth']:.3f}")
    print(f"  Median final wealth: ${res['median_final_wealth']:.3f}")
    print(f"  Geometric mean return: {res['geometric_mean_return']*100:.2f}%")
    print(f"  Probability of profit: {res['prob_profit']*100:.1f}%")
    print(f"  Sharpe ratio: {res['sharpe_ratio']:.3f}")

## 5. Visualize Wealth Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Sample wealth paths
ax = axes[0, 0]
for name, res in results.items():
    # Plot 5 sample paths
    for i in range(5):
        ax.plot(res['wealth_paths'][i], alpha=0.3)
ax.set_title('Sample Wealth Paths (5 paths per strategy)', fontsize=12, fontweight='bold')
ax.set_xlabel('Time Period')
ax.set_ylabel('Wealth')
ax.legend([k for k in strategies.keys() for _ in range(5)][:20], fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Final wealth distributions
ax = axes[0, 1]
for name, res in results.items():
    final_wealth = res['wealth_paths'][:, -1]
    ax.hist(final_wealth, bins=50, alpha=0.5, label=name)
ax.set_title('Final Wealth Distributions', fontsize=12, fontweight='bold')
ax.set_xlabel('Final Wealth')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Mean wealth evolution
ax = axes[1, 0]
for name, res in results.items():
    mean_path = np.mean(res['wealth_paths'], axis=0)
    ax.plot(mean_path, linewidth=2, label=name)
ax.set_title('Mean Wealth Evolution', fontsize=12, fontweight='bold')
ax.set_xlabel('Time Period')
ax.set_ylabel('Mean Wealth')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Median wealth evolution
ax = axes[1, 1]
for name, res in results.items():
    median_path = np.median(res['wealth_paths'], axis=0)
    ax.plot(median_path, linewidth=2, label=name)
ax.set_title('Median Wealth Evolution (Kelly optimizes geometric mean)', fontsize=12, fontweight='bold')
ax.set_xlabel('Time Period')
ax.set_ylabel('Median Wealth')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight: Kelly maximizes geometric mean (median), not arithmetic mean!")

## 6. Efficient Frontier vs Kelly

Plot the mean-variance efficient frontier and show where Kelly lies.

In [ ]:
# Compute efficient frontier
frontier = sp_opt.efficient_frontier(n_points=50)

# Calculate strategy positions
strategy_points = {}
for name, weights in strategies.items():
    ret = weights @ sp_opt.mean_returns * 252
    vol = np.sqrt(weights @ sp_opt.cov_matrix @ weights) * np.sqrt(252)
    strategy_points[name] = (vol, ret)

# Plot
plt.figure(figsize=(12, 8))
plt.plot(frontier['volatilities'] * np.sqrt(252), frontier['returns'] * 252, 
         'b-', linewidth=2, label='Efficient Frontier')

colors = {'Mean-Variance': 'red', 'Kelly': 'green', 'Half Kelly': 'orange', 'Equal Weight': 'purple'}
for name, (vol, ret) in strategy_points.items():
    plt.scatter(vol, ret, s=200, c=colors[name], marker='*', 
               edgecolors='black', linewidths=2, label=name, zorder=5)

plt.xlabel('Volatility (Annualized)', fontsize=12)
plt.ylabel('Expected Return (Annualized)', fontsize=12)
plt.title('Mean-Variance Efficient Frontier vs Kelly Criterion', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

print("\nNote: Kelly often lies BELOW the efficient frontier in mean-std space!")
print("This is because Kelly maximizes E[log(1+R)], not E[R] - (λ/2)Var[R]")

## 7. Theoretical Analysis

### Why Kelly differs from Mean-Variance:

**Mean-Variance**: Tangency portfolio is
$$w_{MV} = \frac{1}{\lambda} \Sigma^{-1} \mu$$

**Kelly**: Growth-optimal portfolio maximizes
$$E[\log(1 + w^T r)] \approx w^T \mu - \frac{1}{2} w^T \Sigma w$$

This gives:
$$w_{Kelly} = \Sigma^{-1} \mu$$

So Kelly ≈ Mean-Variance with λ=2!

### Growth Rate Comparison

In [ ]:
# Calculate theoretical growth rates
print("\nTheoretical Growth Rates:")
for name, weights in strategies.items():
    growth_rate = mp_opt.time_series_growth_rate(weights)
    print(f"{name}: {growth_rate * 252 * 100:.2f}% per year")

print("\nKelly maximizes this growth rate by construction!")

## 8. Kelly Leverage Analysis

In [ ]:
# Kelly leverage ratio
leverage = mp_opt.kelly_leverage()
print(f"\nKelly leverage ratio: {leverage:.2f}x")
print(f"Kelly weights sum: {np.sum(kelly_weights):.4f}")
print(f"Mean-Variance weights sum: {np.sum(mv_weights):.4f}")

if leverage > 1.5:
    print("\n⚠️  Kelly suggests significant leverage - fractional Kelly may be prudent!")

## Key Takeaways

1. **Single-period**: Mean-variance and Kelly can give similar results, especially for moderate risk aversion (λ≈2)

2. **Multi-period**: Kelly maximizes geometric mean growth rate, which equals median terminal wealth

3. **Mean vs Median**: Kelly may underperform mean-variance in arithmetic mean but outperforms in geometric mean

4. **Leverage**: Full Kelly can be aggressive - fractional Kelly (e.g., 1/2 Kelly) reduces volatility

5. **Long-run**: Kelly is asymptotically optimal - it will beat any other strategy almost surely in the long run

6. **Practical**: Many practitioners use fractional Kelly (25-50%) to balance growth and risk